In [2]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.39


In [3]:
ws = Workspace.from_config()

exp = Experiment(workspace=ws, name='estimator-test')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [5]:
cpu_cluster = ws.compute_targets['prodmod-compute1']

# use get_status() to get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-05-20T23:26:07.076000+00:00', 'errors': None, 'creationTime': '2019-05-20T21:19:08.973390+00:00', 'modifiedTime': '2019-05-20T21:20:48.008942+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT600S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_D2_V2'}


In [6]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

prodmod-compute1 AmlCompute Succeeded


In [7]:
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

In [8]:
script_params = {'--numbers-in-sequence': 10}

In [9]:
est = Estimator(source_directory='.',
               script_params=script_params,
               compute_target='local',
               entry_script='dummy_train.py',
               use_docker=False)

In [10]:
run = exp.submit(est)

In [11]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Using Docker

In [12]:
from azureml.core.container_registry import ContainerRegistry

In [13]:
image_name = 'continuumio/miniconda3'

In [19]:
ws.get_details()

{'id': '/subscriptions/3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d/resourceGroups/prodmod_rg/providers/Microsoft.MachineLearningServices/workspaces/prodmod',
 'name': 'prodmod',
 'location': 'westus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '60b5573d-c2c7-4ff1-bd7b-b0d4966cdc90',
 'description': '',
 'friendlyName': 'prodmod',
 'creationTime': '2019-05-20T21:03:06.5153700+00:00',
 'containerRegistry': '/subscriptions/3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d/resourceGroups/prodmod_rg/providers/Microsoft.ContainerRegistry/registries/prodmod3d2bdaf0',
 'keyVault': '/subscriptions/3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d/resourcegroups/prodmod_rg/providers/microsoft.keyvault/vaults/prodmodkeyvault4dffdcce4',
 'applicationInsights': '/subscriptions/3196c5e8-a6ed-496e-a9dc-bfa78ce96d7d/resourcegroups/prodmod_rg/providers/microsoft.insights/components/prodmodinsightscc2312924',
 'identityPrincipalId': '7c45ae4c-f97c-4e6b-ba0d-446883f98526',
 'identityTenantId': 'bc63996d-bc

In [22]:
user_managed_dependencies = True

In [23]:
script_params = {
    '--numbers-in-sequence': 10
}

In [24]:
est = Estimator(source_directory='.', 
                script_params=script_params, 
                compute_target='local', 
                entry_script='dummy_train.py',
                custom_docker_image=image_name,
                # uncomment below line to use your private ACR
                #image_registry_details=image_registry_details,
                user_managed=user_managed_dependencies
                )

In [25]:
run = exp.submit(est)
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Intelligent Hyperparameter tuning

In [27]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice

In [28]:
ps = RandomParameterSampling(
    {
        '--numbers-in-sequence': choice(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20)
    }
)

In [29]:
est = Estimator(source_directory='.', script_params={}, compute_target=cpu_cluster, entry_script='dummy_train.py', use_docker=True)

In [30]:

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

In [31]:
hdc = HyperDriveConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='Fibonacci numbers', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=10,
                          max_concurrent_runs=4)

In [32]:
hdr = exp.submit(config=hdc)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [34]:
best_run = hdr.get_best_run_by_primary_metric()